# Projeto Grupo BT-G3


## **Integrantes do grupo**
- Daniel Barzilai
- Larissa Carvalho
- Maria Luisa Maia
- Pedro Rezende
- Rafael Moritz
- Vitor Oliveira

<center><img src="https://www.inteli.edu.br/wp-content/uploads/2021/08/20172028/marca_1-2.png" width="50%" height="50%"/></center>

<h1 align='center'><b>IA para Marketing: Monitoramento de campanhas utilizando processamento de linguagem natural (PLN)<b></h1>

<center><img src="https://upload.wikimedia.org/wikipedia/commons/c/c2/Btg-logo-blue.svg" width="50%" height="50%"/></center>

<h2 align='center'>O Banco BTG Pactual enfrenta um desafio na área de Marketing em entender as necessidades e demandas dos clientes de maneira fácil e rápida nas redes sociais. A solução proposta para esse problema foi o desenvolvimento de uma Inteligência Artificial utilizando processamento de linguagem natural (PLN), capaz de monitorar as campanhas de marketing, voltadas para o Instagram. O objetivo principal dessa solução é rastrear os dados em tempo real, analisar e interpretar as mensagens e comentários enviados pelos clientes na rede social, a fim de identificar as necessidades e demandas de forma precisa e eficiente.</h2>

---

# Sobre os dados

Esse projeto está utilizando dados coletados e tratados pela equipe de Automation do BTG Pactual, o qual disponibilizou o dataset. Com base nas informações dispostas nesse dataset, realizaremos insights a cerca dos comentários feitos nos posts do Instagram do próprio banco. Vale lembrar que os dados estão anonimizados e resguardados para manter a privacidade e ética com os usuários e com o banco.

# 1. Instalação / Setup

Para o início do projeto, fizemos o desenvolvimento no Google Colab, por isso temos uma célula de conexão com o Google Drive, para poder acessar os dados. Caso seja rodado no Jupyter Notebook, precisará do dataset baixado.

In [1]:
#Conectar com o Google Drive

from google.colab import drive
drive.mount('/content/drive')

#Conectando o ambiente ao Google Drive

Mounted at /content/drive


Aqui nós fazemos as importações para tratamento dos dados, pré-processamento dos dados e modelamento do Bag of Words. 

## pips

In [2]:
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 64.7 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.2
    Uninstalling spacy-3.5.2:
      Successfully uninstalled spacy-3.5.2


In [3]:
!pip install wordcloud

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Imports

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import re

from sklearn.feature_extraction.text import CountVectorizer
import ast
from keras.preprocessing.text import Tokenizer
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential 
from keras.layers import Dense, Embedding, GlobalMaxPooling1D, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from keras.utils import pad_sequences

import nltk
import spacy
import gensim
import pickle
from scipy.spatial.distance import cosine
from gensim.models import KeyedVectors

O desenvolvimento da rede neural seguiu o tutorial abaixo:
https://medium.com/@jvsavietto6/classificando-texto-com-redes-neurais-artificiais-150ef448b13d

# 2. Entendimento e Tratamento dos Dados

Rodando o dataset, para analisar seu conteúdo:

In [8]:
df = pd.read_csv('/content/drive/MyDrive/Módulo 6/Sprint 4 - Projeto/nova_base_tratada')
df

,autor,sentimento,texto_tratado
0,winthegame_of,1,"['alvarez', 'marsal', 'estar', 'conosco', 'spo..."
1,marta_bego,1,"['btgpactual', 'with', 'make_repost', 'entenda..."
2,lmviapiana,2,"['minuto', 'touro', 'ouro']"
3,ricktolledo,2,"['querer', 'saber', 'banking', 'administrar']"
4,donadabolsaoficial,0,"['início', 'guerra', 'rússia', 'ucrânia', 'vel..."
...,...,...,...
8035,perspectiveinvestimentos,2,"['excelente', 'explicação']"
8036,eduardocolares,2,"['atendar', 'telefone', 'amor', 'deus']"
8037,danielucm,2,"['grande', 'fiis', 'mercado', 'não', 'selecion..."
8038,amgcapitalinvest,1,"['erro', 'financeiro', 'eliminar', 'parar', 'o..."


In [9]:
df.columns

Index(['autor', 'sentimento', 'texto_tratado'], dtype='object')

In [10]:
df['texto_tratado']

0       ['alvarez', 'marsal', 'estar', 'conosco', 'spo...
1       ['btgpactual', 'with', 'make_repost', 'entenda...
2                             ['minuto', 'touro', 'ouro']
3           ['querer', 'saber', 'banking', 'administrar']
4       ['início', 'guerra', 'rússia', 'ucrânia', 'vel...
                              ...                        
8035                          ['excelente', 'explicação']
8036              ['atendar', 'telefone', 'amor', 'deus']
8037    ['grande', 'fiis', 'mercado', 'não', 'selecion...
8038    ['erro', 'financeiro', 'eliminar', 'parar', 'o...
8039    ['morning', 'call', 'não', 'aparecer', 'spotif...
Name: texto_tratado, Length: 8040, dtype: object

In [11]:
# Supondo que seu DataFrame seja chamado de df e a coluna seja 'texto_tratado'
df['texto_tratado'] = df['texto_tratado'].str.replace("'", "")
df['texto_tratado']

0       [alvarez, marsal, estar, conosco, sportainmet,...
1       [btgpactual, with, make_repost, entendar, impa...
2                                   [minuto, touro, ouro]
3                   [querer, saber, banking, administrar]
4       [início, guerra, rússia, ucrânia, velho, apare...
                              ...                        
8035                              [excelente, explicação]
8036                      [atendar, telefone, amor, deus]
8037    [grande, fiis, mercado, não, selecionar, princ...
8038    [erro, financeiro, eliminar, parar, ostentar, ...
8039    [morning, call, não, aparecer, spotify, atuali...
Name: texto_tratado, Length: 8040, dtype: object

# 10. Word2Vec com CBOW

## Estruturação

In [12]:
df

,autor,sentimento,texto_tratado
0,winthegame_of,1,"[alvarez, marsal, estar, conosco, sportainmet,..."
1,marta_bego,1,"[btgpactual, with, make_repost, entendar, impa..."
2,lmviapiana,2,"[minuto, touro, ouro]"
3,ricktolledo,2,"[querer, saber, banking, administrar]"
4,donadabolsaoficial,0,"[início, guerra, rússia, ucrânia, velho, apare..."
...,...,...,...
8035,perspectiveinvestimentos,2,"[excelente, explicação]"
8036,eduardocolares,2,"[atendar, telefone, amor, deus]"
8037,danielucm,2,"[grande, fiis, mercado, não, selecionar, princ..."
8038,amgcapitalinvest,1,"[erro, financeiro, eliminar, parar, ostentar, ..."


In [13]:
cbow = '/content/drive/MyDrive/Módulo 6/Semana 5/cbow_s50/cbow_s50.txt'

In [14]:
model_cbow = KeyedVectors.load_word2vec_format(cbow)

## Teste isolado

In [15]:
# Testando o word2vec
wordvec_test = model_cbow['projeto']

wordvec_test

array([-0.074174, -0.152088,  0.086627, -0.224567,  0.362562,  0.130683,
       -0.089179, -0.086973,  0.309501,  0.004112, -0.308202,  0.351789,
       -0.477863,  0.050276,  0.213283,  0.159895, -0.285545, -0.08832 ,
       -0.015449,  0.014816, -0.613861,  0.502556,  0.021688,  0.369492,
        0.280691,  0.016868,  0.105584, -0.180754, -0.078456,  0.148032,
        0.36293 , -0.011634,  0.412191, -0.009049,  0.010404,  0.131242,
       -0.032483, -0.133067, -0.063802,  0.434015, -0.214768, -0.072132,
        0.045601, -0.368866,  0.502808,  0.048293, -0.254894,  0.142581,
       -0.075066,  0.015646], dtype=float32)

## Definição de função

In [16]:
def create_sentence_vector(model, df):
    sentence_table = []
    for sentence in df['texto_tratado']:
        word_vectors = [model[word] for word in sentence if word in model]
        if len(word_vectors) > 0:
            sentence_vector = sum(word_vectors) / len(word_vectors)
        else:
            sentence_vector = [None] * 100  # Cria uma lista de 100 elementos None
        sentence_table.append((sentence, *sentence_vector[:50]))  # Adiciona apenas os primeiros 50 elementos do vetor

    column_labels = ['Frase']
    for i in range(50):
        column_labels.append(f'Vetor{i+1}')
    df_vec = pd.DataFrame(sentence_table, columns=column_labels)

    df["sentimentoNumerico"] = df["sentimento"].replace({'NEGATIVE': -1, 'POSITIVE': 1, 'NEUTRAL': 0})

    # Definir o índice do DataFrame df_vec como o mesmo índice de df_processada['sentimentoNumerico']
    df_vec.set_index(df["sentimentoNumerico"].index, inplace=True)

    df_vec['sentimento'] = df["sentimentoNumerico"]
    df_vec = df_vec.dropna()

    return df_vec

## Teste de funções

In [17]:
df_vec = create_sentence_vector(model_cbow, df)
df_vec

,Frase,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,Vetor8,Vetor9,...,Vetor42,Vetor43,Vetor44,Vetor45,Vetor46,Vetor47,Vetor48,Vetor49,Vetor50,sentimento
0,"[alvarez, marsal, estar, conosco, sportainmet,...",0.214823,-0.123036,0.220707,-0.079739,-0.019442,0.202300,0.073742,0.042477,0.057268,...,0.012086,-0.096122,0.148153,0.092629,-0.027504,0.149860,-0.029617,0.032065,0.190372,1
1,"[btgpactual, with, make_repost, entendar, impa...",0.220860,-0.123713,0.205813,-0.063700,-0.010433,0.195687,0.072226,0.026175,0.058045,...,0.006447,-0.076174,0.157446,0.085306,-0.001963,0.156306,-0.019182,0.035594,0.192635,1
2,"[minuto, touro, ouro]",0.265227,-0.068285,0.152235,-0.044329,-0.102729,0.141353,0.092800,0.113174,0.015783,...,0.078032,-0.202677,0.155750,0.062291,0.007038,0.134573,0.014635,0.034189,0.345674,2
3,"[querer, saber, banking, administrar]",0.189241,-0.169407,0.281181,-0.093912,-0.001968,0.181633,0.080923,0.055674,0.103454,...,0.092738,-0.160601,0.179499,0.091803,-0.063762,0.171190,-0.043618,0.030563,0.196250,2
4,"[início, guerra, rússia, ucrânia, velho, apare...",0.219838,-0.138311,0.229412,-0.057420,-0.014510,0.212875,0.058865,0.068096,0.060398,...,0.040836,-0.112662,0.170401,0.094290,-0.046000,0.168181,-0.018029,0.027667,0.180134,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8035,"[excelente, explicação]",0.190917,-0.133475,0.241675,-0.053180,0.067256,0.201138,0.034109,-0.078718,-0.066131,...,-0.082151,0.016113,0.154861,0.068700,-0.004302,0.079717,-0.028388,-0.017448,0.188785,2
8036,"[atendar, telefone, amor, deus]",0.188641,-0.119377,0.199339,-0.105448,0.023176,0.178837,0.069476,-0.004494,0.034710,...,0.034035,-0.126673,0.165176,0.080313,-0.024160,0.118848,-0.003502,0.087053,0.215656,2
8037,"[grande, fiis, mercado, não, selecionar, princ...",0.219437,-0.141124,0.217854,-0.065569,-0.020409,0.205403,0.061334,0.041665,0.063050,...,0.031438,-0.099000,0.176588,0.092546,-0.046104,0.145734,-0.019857,0.029422,0.183504,2
8038,"[erro, financeiro, eliminar, parar, ostentar, ...",0.218153,-0.146420,0.245234,-0.056113,-0.036187,0.224789,0.074320,0.045111,0.040841,...,0.035260,-0.126793,0.171115,0.095479,-0.033473,0.158747,-0.006721,0.029326,0.198914,1


In [18]:
#df_vec.to_csv('Word2Vec_Cbow_modelo_treinado',encoding='utf-8', index=False, header=True)

# 11. Naive Bayes + Word2Vec com CBOW

In [19]:
label = preprocessing.LabelEncoder()

In [20]:
label.fit(df_vec['sentimento'])
df_vec['sentimento'] = label.transform(df_vec['sentimento'])

In [21]:
df_vec = df_vec.dropna()
df_vec

,Frase,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,Vetor8,Vetor9,...,Vetor42,Vetor43,Vetor44,Vetor45,Vetor46,Vetor47,Vetor48,Vetor49,Vetor50,sentimento
0,"[alvarez, marsal, estar, conosco, sportainmet,...",0.214823,-0.123036,0.220707,-0.079739,-0.019442,0.202300,0.073742,0.042477,0.057268,...,0.012086,-0.096122,0.148153,0.092629,-0.027504,0.149860,-0.029617,0.032065,0.190372,1
1,"[btgpactual, with, make_repost, entendar, impa...",0.220860,-0.123713,0.205813,-0.063700,-0.010433,0.195687,0.072226,0.026175,0.058045,...,0.006447,-0.076174,0.157446,0.085306,-0.001963,0.156306,-0.019182,0.035594,0.192635,1
2,"[minuto, touro, ouro]",0.265227,-0.068285,0.152235,-0.044329,-0.102729,0.141353,0.092800,0.113174,0.015783,...,0.078032,-0.202677,0.155750,0.062291,0.007038,0.134573,0.014635,0.034189,0.345674,2
3,"[querer, saber, banking, administrar]",0.189241,-0.169407,0.281181,-0.093912,-0.001968,0.181633,0.080923,0.055674,0.103454,...,0.092738,-0.160601,0.179499,0.091803,-0.063762,0.171190,-0.043618,0.030563,0.196250,2
4,"[início, guerra, rússia, ucrânia, velho, apare...",0.219838,-0.138311,0.229412,-0.057420,-0.014510,0.212875,0.058865,0.068096,0.060398,...,0.040836,-0.112662,0.170401,0.094290,-0.046000,0.168181,-0.018029,0.027667,0.180134,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8035,"[excelente, explicação]",0.190917,-0.133475,0.241675,-0.053180,0.067256,0.201138,0.034109,-0.078718,-0.066131,...,-0.082151,0.016113,0.154861,0.068700,-0.004302,0.079717,-0.028388,-0.017448,0.188785,2
8036,"[atendar, telefone, amor, deus]",0.188641,-0.119377,0.199339,-0.105448,0.023176,0.178837,0.069476,-0.004494,0.034710,...,0.034035,-0.126673,0.165176,0.080313,-0.024160,0.118848,-0.003502,0.087053,0.215656,2
8037,"[grande, fiis, mercado, não, selecionar, princ...",0.219437,-0.141124,0.217854,-0.065569,-0.020409,0.205403,0.061334,0.041665,0.063050,...,0.031438,-0.099000,0.176588,0.092546,-0.046104,0.145734,-0.019857,0.029422,0.183504,2
8038,"[erro, financeiro, eliminar, parar, ostentar, ...",0.218153,-0.146420,0.245234,-0.056113,-0.036187,0.224789,0.074320,0.045111,0.040841,...,0.035260,-0.126793,0.171115,0.095479,-0.033473,0.158747,-0.006721,0.029326,0.198914,1


## Separando Treino e Teste

In [22]:
target = df_vec['sentimento']

In [23]:
feature = df_vec.iloc[:,1:50]

In [24]:
feature

,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,Vetor8,Vetor9,Vetor10,...,Vetor40,Vetor41,Vetor42,Vetor43,Vetor44,Vetor45,Vetor46,Vetor47,Vetor48,Vetor49
0,0.214823,-0.123036,0.220707,-0.079739,-0.019442,0.202300,0.073742,0.042477,0.057268,0.168187,...,0.076954,-0.281536,0.012086,-0.096122,0.148153,0.092629,-0.027504,0.149860,-0.029617,0.032065
1,0.220860,-0.123713,0.205813,-0.063700,-0.010433,0.195687,0.072226,0.026175,0.058045,0.163238,...,0.086821,-0.305640,0.006447,-0.076174,0.157446,0.085306,-0.001963,0.156306,-0.019182,0.035594
2,0.265227,-0.068285,0.152235,-0.044329,-0.102729,0.141353,0.092800,0.113174,0.015783,0.202198,...,-0.008447,-0.193025,0.078032,-0.202677,0.155750,0.062291,0.007038,0.134573,0.014635,0.034189
3,0.189241,-0.169407,0.281181,-0.093912,-0.001968,0.181633,0.080923,0.055674,0.103454,0.144342,...,0.060960,-0.254610,0.092738,-0.160601,0.179499,0.091803,-0.063762,0.171190,-0.043618,0.030563
4,0.219838,-0.138311,0.229412,-0.057420,-0.014510,0.212875,0.058865,0.068096,0.060398,0.156307,...,0.067551,-0.269136,0.040836,-0.112662,0.170401,0.094290,-0.046000,0.168181,-0.018029,0.027667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8035,0.190917,-0.133475,0.241675,-0.053180,0.067256,0.201138,0.034109,-0.078718,-0.066131,0.187608,...,0.014565,-0.321192,-0.082151,0.016113,0.154861,0.068700,-0.004302,0.079717,-0.028388,-0.017448
8036,0.188641,-0.119377,0.199339,-0.105448,0.023176,0.178837,0.069476,-0.004494,0.034710,0.150081,...,0.071114,-0.194663,0.034035,-0.126673,0.165176,0.080313,-0.024160,0.118848,-0.003502,0.087053
8037,0.219437,-0.141124,0.217854,-0.065569,-0.020409,0.205403,0.061334,0.041665,0.063050,0.167985,...,0.077558,-0.300234,0.031438,-0.099000,0.176588,0.092546,-0.046104,0.145734,-0.019857,0.029422
8038,0.218153,-0.146420,0.245234,-0.056113,-0.036187,0.224789,0.074320,0.045111,0.040841,0.150971,...,0.060317,-0.270532,0.035260,-0.126793,0.171115,0.095479,-0.033473,0.158747,-0.006721,0.029326


In [25]:
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.2, random_state=42)

## Avaliação do modelo 

In [26]:
clf = GaussianNB()

clf = clf.fit(X_train,y_train.values.ravel())

Y_pred = clf.predict(X_test)

print(classification_report(y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.25      0.73      0.37       360
           1       0.51      0.30      0.38       597
           2       0.32      0.10      0.15       651

    accuracy                           0.31      1608
   macro avg       0.36      0.38      0.30      1608
weighted avg       0.38      0.31      0.28      1608



In [27]:
acc_score = accuracy_score(y_test, Y_pred)
format_output = "{:.2%}".format(acc_score)
print("Precisão final de :",format_output) 

Precisão final de : 31.47%


# 12. Word2Vec com embedding layer

## Definição de função

In [28]:
from gensim.models import Word2Vec

# Função que treina o modelo Word2Vec no corpus do dataframe
def train_word2vec(df, column_name):
    # Obtém as frases tokenizadas
    sentences = df[column_name].tolist()
    
    # Treina o modelo Word2Vec
    model = Word2Vec(sentences, min_count=1)
    
    return model

In [29]:
# Função que define os vetores para cada palavra do vocabulario
def get_word_vectors(model, sentence):
    vectors = []
    for word in sentence:
        if word in model.wv:
            vectors.append(model.wv[word]) # Append na lista de vetores
    if vectors:
        return np.sum(vectors, axis=0)/len(sentence) # Soma dos vetores para cada frase
    else:
        return np.zeros(model.vector_size)

# Criação do dataframe de vetores para cada frase
def create_word2vec_dataframe(df, column_name, model):
    sentences = df[column_name].tolist()
    vectors = [get_word_vectors(model, sentence) for sentence in sentences] # Itera para cada frase um vetor
    # Criação do dataframe
    df_vectors = pd.DataFrame(vectors, columns=[f"Vetor{i}" for i in range(model.vector_size)])
    df_word2vec = pd.concat([df, df_vectors], axis=1)
    return df_word2vec

## Teste de funções

In [30]:
model = train_word2vec(df, 'texto_tratado')

In [31]:
df_word2vec = create_word2vec_dataframe(df,'texto_tratado', model)
df_word2vec

,autor,sentimento,texto_tratado,sentimentoNumerico,Vetor0,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,...,Vetor90,Vetor91,Vetor92,Vetor93,Vetor94,Vetor95,Vetor96,Vetor97,Vetor98,Vetor99
0,winthegame_of,1,"[alvarez, marsal, estar, conosco, sportainmet,...",1,0.067558,0.138432,-0.492918,-0.384604,0.446953,0.095563,...,-0.244758,-0.163760,-0.271157,0.003377,-0.210831,0.011274,0.477493,-0.116411,0.122214,-0.022393
1,marta_bego,1,"[btgpactual, with, make_repost, entendar, impa...",1,0.059862,0.136326,-0.486257,-0.342627,0.413020,0.111388,...,-0.250583,-0.157099,-0.272688,-0.003315,-0.209944,-0.006222,0.442775,-0.077177,0.113582,0.002142
2,lmviapiana,2,"[minuto, touro, ouro]",2,-0.024142,0.206002,-0.553528,-0.423587,0.428873,0.037835,...,-0.141497,-0.179339,-0.349158,-0.067359,-0.136633,0.021721,0.533516,-0.186150,0.197291,-0.010571
3,ricktolledo,2,"[querer, saber, banking, administrar]",2,0.062537,0.139655,-0.418685,-0.347889,0.403975,0.060553,...,-0.226398,-0.162073,-0.202206,-0.043939,-0.162619,0.076544,0.446086,-0.079397,0.092905,0.018417
4,donadabolsaoficial,0,"[início, guerra, rússia, ucrânia, velho, apare...",0,0.034056,0.179664,-0.517268,-0.385962,0.435429,0.121181,...,-0.257256,-0.166135,-0.248145,-0.009300,-0.243360,-0.032225,0.487327,-0.069010,0.160396,0.010792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8035,perspectiveinvestimentos,2,"[excelente, explicação]",2,0.011201,0.170449,-0.493170,-0.416638,0.418993,0.056864,...,-0.246850,-0.190740,-0.152463,-0.045937,-0.204091,-0.092101,0.506569,0.030068,0.092457,-0.002814
8036,eduardocolares,2,"[atendar, telefone, amor, deus]",2,0.053721,0.209818,-0.553408,-0.389849,0.428691,0.141916,...,-0.256920,-0.186227,-0.269119,-0.055570,-0.248289,-0.027219,0.480121,-0.057997,0.154438,0.013144
8037,danielucm,2,"[grande, fiis, mercado, não, selecionar, princ...",2,0.037945,0.167857,-0.494652,-0.380031,0.422713,0.143329,...,-0.239438,-0.165142,-0.245146,-0.001706,-0.282409,-0.028503,0.466386,-0.045153,0.156707,0.035785
8038,amgcapitalinvest,1,"[erro, financeiro, eliminar, parar, ostentar, ...",1,0.038255,0.186313,-0.512019,-0.375521,0.432356,0.140662,...,-0.251595,-0.185266,-0.275083,-0.017809,-0.254864,-0.014637,0.462504,-0.064120,0.133442,0.026779


In [32]:
df_word2vec = df_word2vec.drop(columns=['autor', 'sentimento'])
df_word2vec

,texto_tratado,sentimentoNumerico,Vetor0,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,...,Vetor90,Vetor91,Vetor92,Vetor93,Vetor94,Vetor95,Vetor96,Vetor97,Vetor98,Vetor99
0,"[alvarez, marsal, estar, conosco, sportainmet,...",1,0.067558,0.138432,-0.492918,-0.384604,0.446953,0.095563,-0.016932,-0.437999,...,-0.244758,-0.163760,-0.271157,0.003377,-0.210831,0.011274,0.477493,-0.116411,0.122214,-0.022393
1,"[btgpactual, with, make_repost, entendar, impa...",1,0.059862,0.136326,-0.486257,-0.342627,0.413020,0.111388,-0.057893,-0.422980,...,-0.250583,-0.157099,-0.272688,-0.003315,-0.209944,-0.006222,0.442775,-0.077177,0.113582,0.002142
2,"[minuto, touro, ouro]",2,-0.024142,0.206002,-0.553528,-0.423587,0.428873,0.037835,-0.001885,-0.422896,...,-0.141497,-0.179339,-0.349158,-0.067359,-0.136633,0.021721,0.533516,-0.186150,0.197291,-0.010571
3,"[querer, saber, banking, administrar]",2,0.062537,0.139655,-0.418685,-0.347889,0.403975,0.060553,-0.019941,-0.344664,...,-0.226398,-0.162073,-0.202206,-0.043939,-0.162619,0.076544,0.446086,-0.079397,0.092905,0.018417
4,"[início, guerra, rússia, ucrânia, velho, apare...",0,0.034056,0.179664,-0.517268,-0.385962,0.435429,0.121181,-0.017307,-0.445177,...,-0.257256,-0.166135,-0.248145,-0.009300,-0.243360,-0.032225,0.487327,-0.069010,0.160396,0.010792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8035,"[excelente, explicação]",2,0.011201,0.170449,-0.493170,-0.416638,0.418993,0.056864,-0.005847,-0.434418,...,-0.246850,-0.190740,-0.152463,-0.045937,-0.204091,-0.092101,0.506569,0.030068,0.092457,-0.002814
8036,"[atendar, telefone, amor, deus]",2,0.053721,0.209818,-0.553408,-0.389849,0.428691,0.141916,-0.027666,-0.493831,...,-0.256920,-0.186227,-0.269119,-0.055570,-0.248289,-0.027219,0.480121,-0.057997,0.154438,0.013144
8037,"[grande, fiis, mercado, não, selecionar, princ...",2,0.037945,0.167857,-0.494652,-0.380031,0.422713,0.143329,-0.035665,-0.456224,...,-0.239438,-0.165142,-0.245146,-0.001706,-0.282409,-0.028503,0.466386,-0.045153,0.156707,0.035785
8038,"[erro, financeiro, eliminar, parar, ostentar, ...",1,0.038255,0.186313,-0.512019,-0.375521,0.432356,0.140662,-0.002691,-0.465539,...,-0.251595,-0.185266,-0.275083,-0.017809,-0.254864,-0.014637,0.462504,-0.064120,0.133442,0.026779


# 13. Naive Bayes + Word2Vec com embedding layer

In [33]:
df_word2vec = df_word2vec.dropna()
df_word2vec

,texto_tratado,sentimentoNumerico,Vetor0,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,...,Vetor90,Vetor91,Vetor92,Vetor93,Vetor94,Vetor95,Vetor96,Vetor97,Vetor98,Vetor99
0,"[alvarez, marsal, estar, conosco, sportainmet,...",1,0.067558,0.138432,-0.492918,-0.384604,0.446953,0.095563,-0.016932,-0.437999,...,-0.244758,-0.163760,-0.271157,0.003377,-0.210831,0.011274,0.477493,-0.116411,0.122214,-0.022393
1,"[btgpactual, with, make_repost, entendar, impa...",1,0.059862,0.136326,-0.486257,-0.342627,0.413020,0.111388,-0.057893,-0.422980,...,-0.250583,-0.157099,-0.272688,-0.003315,-0.209944,-0.006222,0.442775,-0.077177,0.113582,0.002142
2,"[minuto, touro, ouro]",2,-0.024142,0.206002,-0.553528,-0.423587,0.428873,0.037835,-0.001885,-0.422896,...,-0.141497,-0.179339,-0.349158,-0.067359,-0.136633,0.021721,0.533516,-0.186150,0.197291,-0.010571
3,"[querer, saber, banking, administrar]",2,0.062537,0.139655,-0.418685,-0.347889,0.403975,0.060553,-0.019941,-0.344664,...,-0.226398,-0.162073,-0.202206,-0.043939,-0.162619,0.076544,0.446086,-0.079397,0.092905,0.018417
4,"[início, guerra, rússia, ucrânia, velho, apare...",0,0.034056,0.179664,-0.517268,-0.385962,0.435429,0.121181,-0.017307,-0.445177,...,-0.257256,-0.166135,-0.248145,-0.009300,-0.243360,-0.032225,0.487327,-0.069010,0.160396,0.010792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8035,"[excelente, explicação]",2,0.011201,0.170449,-0.493170,-0.416638,0.418993,0.056864,-0.005847,-0.434418,...,-0.246850,-0.190740,-0.152463,-0.045937,-0.204091,-0.092101,0.506569,0.030068,0.092457,-0.002814
8036,"[atendar, telefone, amor, deus]",2,0.053721,0.209818,-0.553408,-0.389849,0.428691,0.141916,-0.027666,-0.493831,...,-0.256920,-0.186227,-0.269119,-0.055570,-0.248289,-0.027219,0.480121,-0.057997,0.154438,0.013144
8037,"[grande, fiis, mercado, não, selecionar, princ...",2,0.037945,0.167857,-0.494652,-0.380031,0.422713,0.143329,-0.035665,-0.456224,...,-0.239438,-0.165142,-0.245146,-0.001706,-0.282409,-0.028503,0.466386,-0.045153,0.156707,0.035785
8038,"[erro, financeiro, eliminar, parar, ostentar, ...",1,0.038255,0.186313,-0.512019,-0.375521,0.432356,0.140662,-0.002691,-0.465539,...,-0.251595,-0.185266,-0.275083,-0.017809,-0.254864,-0.014637,0.462504,-0.064120,0.133442,0.026779


## Separando Treino e Teste

In [34]:
target = df_word2vec['sentimentoNumerico']

In [35]:
feature = df_word2vec.iloc[:,2:102]

In [36]:
feature

,Vetor0,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,Vetor8,Vetor9,...,Vetor90,Vetor91,Vetor92,Vetor93,Vetor94,Vetor95,Vetor96,Vetor97,Vetor98,Vetor99
0,0.067558,0.138432,-0.492918,-0.384604,0.446953,0.095563,-0.016932,-0.437999,-0.394902,0.138984,...,-0.244758,-0.163760,-0.271157,0.003377,-0.210831,0.011274,0.477493,-0.116411,0.122214,-0.022393
1,0.059862,0.136326,-0.486257,-0.342627,0.413020,0.111388,-0.057893,-0.422980,-0.369369,0.153205,...,-0.250583,-0.157099,-0.272688,-0.003315,-0.209944,-0.006222,0.442775,-0.077177,0.113582,0.002142
2,-0.024142,0.206002,-0.553528,-0.423587,0.428873,0.037835,-0.001885,-0.422896,-0.552002,0.280266,...,-0.141497,-0.179339,-0.349158,-0.067359,-0.136633,0.021721,0.533516,-0.186150,0.197291,-0.010571
3,0.062537,0.139655,-0.418685,-0.347889,0.403975,0.060553,-0.019941,-0.344664,-0.317722,0.171924,...,-0.226398,-0.162073,-0.202206,-0.043939,-0.162619,0.076544,0.446086,-0.079397,0.092905,0.018417
4,0.034056,0.179664,-0.517268,-0.385962,0.435429,0.121181,-0.017307,-0.445177,-0.403767,0.167685,...,-0.257256,-0.166135,-0.248145,-0.009300,-0.243360,-0.032225,0.487327,-0.069010,0.160396,0.010792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8035,0.011201,0.170449,-0.493170,-0.416638,0.418993,0.056864,-0.005847,-0.434418,-0.293727,0.143874,...,-0.246850,-0.190740,-0.152463,-0.045937,-0.204091,-0.092101,0.506569,0.030068,0.092457,-0.002814
8036,0.053721,0.209818,-0.553408,-0.389849,0.428691,0.141916,-0.027666,-0.493831,-0.423683,0.220266,...,-0.256920,-0.186227,-0.269119,-0.055570,-0.248289,-0.027219,0.480121,-0.057997,0.154438,0.013144
8037,0.037945,0.167857,-0.494652,-0.380031,0.422713,0.143329,-0.035665,-0.456224,-0.390754,0.182915,...,-0.239438,-0.165142,-0.245146,-0.001706,-0.282409,-0.028503,0.466386,-0.045153,0.156707,0.035785
8038,0.038255,0.186313,-0.512019,-0.375521,0.432356,0.140662,-0.002691,-0.465539,-0.404303,0.183533,...,-0.251595,-0.185266,-0.275083,-0.017809,-0.254864,-0.014637,0.462504,-0.064120,0.133442,0.026779


In [37]:
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.2, random_state=42)

## Avaliação do modelo 

In [38]:
clf = GaussianNB()

clf = clf.fit(X_train,y_train.values.ravel())

Y_pred = clf.predict(X_test)

print(classification_report(y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.25      0.78      0.38       360
           1       0.57      0.26      0.36       597
           2       0.43      0.13      0.20       651

    accuracy                           0.33      1608
   macro avg       0.42      0.39      0.31      1608
weighted avg       0.44      0.33      0.30      1608



In [39]:
acc_score = accuracy_score(y_test, Y_pred)
format_output = "{:.2%}".format(acc_score)
print("Precisão final de :",format_output) 

Precisão final de : 32.65%


# Rede Neural - Sequência de palavras

## Teste isolado

In [40]:
! pip install pad_sequences

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pad_sequences: filename=pad_sequences-0.6.1-py3-none-any.whl size=10199 sha256=c8657d842988d2e97b140a69870b744d00ae565960866b6d34afaafc69189b14
  Stored in directory: /root/.cache/pip/wheels/48/9d/22/0a6305b87a9cc46ccc032060a041c3b59f39ac462f7358997e
Successfully built pad_sequences


In [41]:
from keras.preprocessing.text import Tokenizer
#from keras_preprocessing.sequence import pad_sequences
from keras.utils import pad_sequences

citacao = [
    "Estudo no Inteli e escolhi o curso de Sistemas de Informação",
    "Estamos fazendo um projeto para o BTG",
    "Estamos no segundo ano de graduação",
    "Somos do grupo BTG3"
]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(citacao)
citacao_token = tokenizer.texts_to_sequences(citacao)

max_length = max([len(z) for z in citacao_token]) + 1
citacao_pad = pad_sequences(citacao_token, maxlen=max_length, padding="post")

## Construção da rede neural + resultados - base tratada

A rede neural abaixo foi desenvolvida com o tutorial mencionado no ínicio do notebook, que, nesse caso o dataframe utilizado é um arquivo com a base que já passou pelo pré processamento. 

### Dataset já treinado

In [42]:
teste_rede_neural = pd.read_csv("/content/drive/MyDrive/Módulo 6/Sprint 4 - Projeto/nova_base_tratada")

In [43]:
teste_rede_neural

,autor,sentimento,texto_tratado
0,winthegame_of,1,"['alvarez', 'marsal', 'estar', 'conosco', 'spo..."
1,marta_bego,1,"['btgpactual', 'with', 'make_repost', 'entenda..."
2,lmviapiana,2,"['minuto', 'touro', 'ouro']"
3,ricktolledo,2,"['querer', 'saber', 'banking', 'administrar']"
4,donadabolsaoficial,0,"['início', 'guerra', 'rússia', 'ucrânia', 'vel..."
...,...,...,...
8035,perspectiveinvestimentos,2,"['excelente', 'explicação']"
8036,eduardocolares,2,"['atendar', 'telefone', 'amor', 'deus']"
8037,danielucm,2,"['grande', 'fiis', 'mercado', 'não', 'selecion..."
8038,amgcapitalinvest,1,"['erro', 'financeiro', 'eliminar', 'parar', 'o..."


### Separação de treino e teste

In [44]:
x, y = teste_rede_neural["texto_tratado"], teste_rede_neural["sentimento"]

labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)

words = ["o", "ao", 'aos', 'os', 'a', 'as', 'e', 'um', 'uma', 
        'ele', 'ela', 'eles', 'elas', 'do', 'da', 'dos', 'das', 
        'de', 'no', 'na', 'nos', 'nas', 'pelo', 'pela', 'pelos', 
        'pelas', 'num', 'numa', 'nuns', 'numas', 'dum', 'duma', 
        'duns', 'dumas']

x_filter = []

for title in x:
  for word in words:
    title = title.replace(word, '')
  x_filter.append(title)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_filter)

vocab = len(tokenizer.word_docs) + 1

x_filter = tokenizer.texts_to_sequences(x_filter)

max_length = max([len(z) for z in x_filter])
x_filter = pad_sequences(x_filter, maxlen=max_length, padding='post')

x_train, x_test, y_train, y_test = train_test_split(x_filter, y, test_size=0.33)

print("Tamanho de x:", len(x_filter))
print("Tamanho de y:", len(y))


Tamanho de x: 8040
Tamanho de y: 8040


### Criação do modelo

In [45]:
model = Sequential()
model.add(Embedding(input_dim=vocab, output_dim=80, input_length=max_length, trainable = True))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.3))
model.add(Dense(units = 3, activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

mc = ModelCheckpoint('weight.best.hdf5', monitor='val_acc', save_best_only=True, mode='max')

model.fit(x_train, y_train, validation_data = (x_test, y_test), batch_size = 32, epochs = 5, callbacks = [mc])

print(model.evaluate(x_test, y_test))

Epoch 1/5
168/169 [============================>.] - ETA: 0s - loss: 1.0501 - accuracy: 0.4500

169/169 [==============================] - 5s 24ms/step - loss: 1.0499 - accuracy: 0.4501 - val_loss: 0.9958 - val_accuracy: 0.4861
Epoch 2/5
168/169 [============================>.] - ETA: 0s - loss: 0.9222 - accuracy: 0.6185

169/169 [==============================] - 6s 35ms/step - loss: 0.9220 - accuracy: 0.6185 - val_loss: 0.8745 - val_accuracy: 0.6458
Epoch 3/5
167/169 [============================>.] - ETA: 0s - loss: 0.7794 - accuracy: 0.7408

169/169 [==============================] - 6s 37ms/step - loss: 0.7782 - accuracy: 0.7416 - val_loss: 0.7868 - val_accuracy: 0.6812
Epoch 4/5
168/169 [============================>.] - ETA: 0s - loss: 0.6543 - accuracy: 0.7906

169/169 [==============================] - 5s 28ms/step - loss: 0.6540 - accuracy: 0.7906 - val_loss: 0.7355 - val_accuracy: 0.6944
Epoch 5/5
169/169 [==============================] - ETA: 0s - loss: 0.5552 - accuracy: 0.8197

83/83 [==============================] - 1s 6ms/step - loss: 0.7100 - accuracy: 0.6971
[0.7099949717521667, 0.6970610618591309]


In [46]:
# Avaliação do modelo
loss, accuracy = model.evaluate(x_test, y_test)
print('Loss:', loss)
print('Accuracy:', accuracy)

83/83 [==============================] - 1s 6ms/step - loss: 0.7100 - accuracy: 0.6971
Loss: 0.7099949717521667
Accuracy: 0.6970610618591309


### Exportação com a biblioteca pickle

In [47]:
with open('redeNeural_base.pkl', 'wb') as arquivo:
    pickle.dump(model, arquivo)
with open('redeNeural_base.pkl', 'rb') as arquivo:
    modelo_redeNeural_base = pickle.load(arquivo)

## Construção da rede neural + resultados - word2vec com cbow

A rede neural abaixo foi desenvolvida com o tutorial mencionado no ínicio do notebook, que, nesse caso o dataframe utilizado é a variável do Word2Vec com o CBoW. 

### Dataframe do word2vec com o cbow


In [48]:
df_vec

,Frase,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,Vetor8,Vetor9,...,Vetor42,Vetor43,Vetor44,Vetor45,Vetor46,Vetor47,Vetor48,Vetor49,Vetor50,sentimento
0,"[alvarez, marsal, estar, conosco, sportainmet,...",0.214823,-0.123036,0.220707,-0.079739,-0.019442,0.202300,0.073742,0.042477,0.057268,...,0.012086,-0.096122,0.148153,0.092629,-0.027504,0.149860,-0.029617,0.032065,0.190372,1
1,"[btgpactual, with, make_repost, entendar, impa...",0.220860,-0.123713,0.205813,-0.063700,-0.010433,0.195687,0.072226,0.026175,0.058045,...,0.006447,-0.076174,0.157446,0.085306,-0.001963,0.156306,-0.019182,0.035594,0.192635,1
2,"[minuto, touro, ouro]",0.265227,-0.068285,0.152235,-0.044329,-0.102729,0.141353,0.092800,0.113174,0.015783,...,0.078032,-0.202677,0.155750,0.062291,0.007038,0.134573,0.014635,0.034189,0.345674,2
3,"[querer, saber, banking, administrar]",0.189241,-0.169407,0.281181,-0.093912,-0.001968,0.181633,0.080923,0.055674,0.103454,...,0.092738,-0.160601,0.179499,0.091803,-0.063762,0.171190,-0.043618,0.030563,0.196250,2
4,"[início, guerra, rússia, ucrânia, velho, apare...",0.219838,-0.138311,0.229412,-0.057420,-0.014510,0.212875,0.058865,0.068096,0.060398,...,0.040836,-0.112662,0.170401,0.094290,-0.046000,0.168181,-0.018029,0.027667,0.180134,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8035,"[excelente, explicação]",0.190917,-0.133475,0.241675,-0.053180,0.067256,0.201138,0.034109,-0.078718,-0.066131,...,-0.082151,0.016113,0.154861,0.068700,-0.004302,0.079717,-0.028388,-0.017448,0.188785,2
8036,"[atendar, telefone, amor, deus]",0.188641,-0.119377,0.199339,-0.105448,0.023176,0.178837,0.069476,-0.004494,0.034710,...,0.034035,-0.126673,0.165176,0.080313,-0.024160,0.118848,-0.003502,0.087053,0.215656,2
8037,"[grande, fiis, mercado, não, selecionar, princ...",0.219437,-0.141124,0.217854,-0.065569,-0.020409,0.205403,0.061334,0.041665,0.063050,...,0.031438,-0.099000,0.176588,0.092546,-0.046104,0.145734,-0.019857,0.029422,0.183504,2
8038,"[erro, financeiro, eliminar, parar, ostentar, ...",0.218153,-0.146420,0.245234,-0.056113,-0.036187,0.224789,0.074320,0.045111,0.040841,...,0.035260,-0.126793,0.171115,0.095479,-0.033473,0.158747,-0.006721,0.029326,0.198914,1


### Separação treino e teste

In [49]:
x, y = df_vec["Frase"], df_vec["sentimento"]

labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)

words = ["o", "ao", 'aos', 'os', 'a', 'as', 'e', 'um', 'uma', 
        'ele', 'ela', 'eles', 'elas', 'do', 'da', 'dos', 'das', 
        'de', 'no', 'na', 'nos', 'nas', 'pelo', 'pela', 'pelos', 
        'pelas', 'num', 'numa', 'nuns', 'numas', 'dum', 'duma', 
        'duns', 'dumas']

x_filter = []

for title in x:
  for word in words:
    title = title.replace(word, '')
  x_filter.append(title)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_filter)

vocab = len(tokenizer.word_docs) + 1

x_filter = tokenizer.texts_to_sequences(x_filter)

max_length = max([len(z) for z in x_filter])
x_filter = pad_sequences(x_filter, maxlen=max_length, padding='post')

x_train, x_test, y_train, y_test = train_test_split(x_filter, y, test_size=0.33)

print("Tamanho de x:", len(x_filter))
print("Tamanho de y:", len(y))


Tamanho de x: 8040
Tamanho de y: 8040


### Criação do modelo

In [50]:
model = Sequential()
model.add(Embedding(input_dim=vocab, output_dim=80, input_length=max_length, trainable = True))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.3))
model.add(Dense(units = 3, activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

mc = ModelCheckpoint('weight.best.hdf5', monitor='val_acc', save_best_only=True, mode='max')

model.fit(x_train, y_train, validation_data = (x_test, y_test), batch_size = 32, epochs = 5, callbacks = [mc])

print(model.evaluate(x_test, y_test))

Epoch 1/5
167/169 [============================>.] - ETA: 0s - loss: 1.0445 - accuracy: 0.4592

169/169 [==============================] - 5s 21ms/step - loss: 1.0440 - accuracy: 0.4593 - val_loss: 0.9901 - val_accuracy: 0.5053
Epoch 2/5
168/169 [============================>.] - ETA: 0s - loss: 0.9162 - accuracy: 0.6257

169/169 [==============================] - 3s 19ms/step - loss: 0.9163 - accuracy: 0.6255 - val_loss: 0.8728 - val_accuracy: 0.6420
Epoch 3/5
169/169 [==============================] - ETA: 0s - loss: 0.7778 - accuracy: 0.7464

169/169 [==============================] - 4s 25ms/step - loss: 0.7778 - accuracy: 0.7464 - val_loss: 0.7910 - val_accuracy: 0.6782
Epoch 4/5
167/169 [============================>.] - ETA: 0s - loss: 0.6585 - accuracy: 0.7942

169/169 [==============================] - 3s 19ms/step - loss: 0.6590 - accuracy: 0.7943 - val_loss: 0.7408 - val_accuracy: 0.6858
Epoch 5/5
167/169 [============================>.] - ETA: 0s - loss: 0.5649 - accuracy: 0.8190

83/83 [==============================] - 0s 2ms/step - loss: 0.7107 - accuracy: 0.6989
[0.7107133865356445, 0.6989449858665466]


In [51]:
# Avaliação do modelo
loss, accuracy = model.evaluate(x_test, y_test)
print('Loss:', loss)
print('Accuracy:', accuracy)

83/83 [==============================] - 0s 2ms/step - loss: 0.7107 - accuracy: 0.6989
Loss: 0.7107133865356445
Accuracy: 0.6989449858665466


### Exportação com a biblioteca pickle

In [52]:
with open('redeNeural_word2vec_cbow.pkl', 'wb') as arquivo:
    pickle.dump(model, arquivo)
with open('redeNeural_word2vec_cbow.pkl', 'rb') as arquivo:
    modelo_redeNeural_word2vec_cbow = pickle.load(arquivo)

## Construção da rede neural + resultados - word2vec com embedding layer

A rede neural abaixo foi desenvolvida com o tutorial mencionado no ínicio do notebook, que, nesse caso o dataframe utilizado é a variável do Word2Vec com Embedding Layer.

### Dataframe do word2vec com embedding layer

In [53]:
df_word2vec

,texto_tratado,sentimentoNumerico,Vetor0,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,...,Vetor90,Vetor91,Vetor92,Vetor93,Vetor94,Vetor95,Vetor96,Vetor97,Vetor98,Vetor99
0,"[alvarez, marsal, estar, conosco, sportainmet,...",1,0.067558,0.138432,-0.492918,-0.384604,0.446953,0.095563,-0.016932,-0.437999,...,-0.244758,-0.163760,-0.271157,0.003377,-0.210831,0.011274,0.477493,-0.116411,0.122214,-0.022393
1,"[btgpactual, with, make_repost, entendar, impa...",1,0.059862,0.136326,-0.486257,-0.342627,0.413020,0.111388,-0.057893,-0.422980,...,-0.250583,-0.157099,-0.272688,-0.003315,-0.209944,-0.006222,0.442775,-0.077177,0.113582,0.002142
2,"[minuto, touro, ouro]",2,-0.024142,0.206002,-0.553528,-0.423587,0.428873,0.037835,-0.001885,-0.422896,...,-0.141497,-0.179339,-0.349158,-0.067359,-0.136633,0.021721,0.533516,-0.186150,0.197291,-0.010571
3,"[querer, saber, banking, administrar]",2,0.062537,0.139655,-0.418685,-0.347889,0.403975,0.060553,-0.019941,-0.344664,...,-0.226398,-0.162073,-0.202206,-0.043939,-0.162619,0.076544,0.446086,-0.079397,0.092905,0.018417
4,"[início, guerra, rússia, ucrânia, velho, apare...",0,0.034056,0.179664,-0.517268,-0.385962,0.435429,0.121181,-0.017307,-0.445177,...,-0.257256,-0.166135,-0.248145,-0.009300,-0.243360,-0.032225,0.487327,-0.069010,0.160396,0.010792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8035,"[excelente, explicação]",2,0.011201,0.170449,-0.493170,-0.416638,0.418993,0.056864,-0.005847,-0.434418,...,-0.246850,-0.190740,-0.152463,-0.045937,-0.204091,-0.092101,0.506569,0.030068,0.092457,-0.002814
8036,"[atendar, telefone, amor, deus]",2,0.053721,0.209818,-0.553408,-0.389849,0.428691,0.141916,-0.027666,-0.493831,...,-0.256920,-0.186227,-0.269119,-0.055570,-0.248289,-0.027219,0.480121,-0.057997,0.154438,0.013144
8037,"[grande, fiis, mercado, não, selecionar, princ...",2,0.037945,0.167857,-0.494652,-0.380031,0.422713,0.143329,-0.035665,-0.456224,...,-0.239438,-0.165142,-0.245146,-0.001706,-0.282409,-0.028503,0.466386,-0.045153,0.156707,0.035785
8038,"[erro, financeiro, eliminar, parar, ostentar, ...",1,0.038255,0.186313,-0.512019,-0.375521,0.432356,0.140662,-0.002691,-0.465539,...,-0.251595,-0.185266,-0.275083,-0.017809,-0.254864,-0.014637,0.462504,-0.064120,0.133442,0.026779


### Separação treino e teste


In [54]:
x, y = df_word2vec["texto_tratado"], df_word2vec["sentimentoNumerico"]

labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)

words = ["o", "ao", 'aos', 'os', 'a', 'as', 'e', 'um', 'uma', 
        'ele', 'ela', 'eles', 'elas', 'do', 'da', 'dos', 'das', 
        'de', 'no', 'na', 'nos', 'nas', 'pelo', 'pela', 'pelos', 
        'pelas', 'num', 'numa', 'nuns', 'numas', 'dum', 'duma', 
        'duns', 'dumas']

x_filter = []

for title in x:
  for word in words:
    title = title.replace(word, '')
  x_filter.append(title)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_filter)

vocab = len(tokenizer.word_docs) + 1

x_filter = tokenizer.texts_to_sequences(x_filter)

max_length = max([len(z) for z in x_filter])
x_filter = pad_sequences(x_filter, maxlen=max_length, padding='post')

x_train, x_test, y_train, y_test = train_test_split(x_filter, y, test_size=0.33)

print("Tamanho de x:", len(x_filter))
print("Tamanho de y:", len(y))


Tamanho de x: 8040
Tamanho de y: 8040


### Criação do modelo

In [55]:
model = Sequential()
model.add(Embedding(input_dim=vocab, output_dim=80, input_length=max_length, trainable = True))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.3))
model.add(Dense(units = 3, activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

mc = ModelCheckpoint('weight.best.hdf5', monitor='val_acc', save_best_only=True, mode='max')

model.fit(x_train, y_train, validation_data = (x_test, y_test), batch_size = 32, epochs = 5, callbacks = [mc])

print(model.evaluate(x_test, y_test))

Epoch 1/5
169/169 [==============================] - ETA: 0s - loss: 1.0411 - accuracy: 0.4586

169/169 [==============================] - 5s 26ms/step - loss: 1.0411 - accuracy: 0.4586 - val_loss: 0.9917 - val_accuracy: 0.5057
Epoch 2/5
169/169 [==============================] - ETA: 0s - loss: 0.9112 - accuracy: 0.6025

169/169 [==============================] - 4s 25ms/step - loss: 0.9112 - accuracy: 0.6025 - val_loss: 0.8761 - val_accuracy: 0.6481
Epoch 3/5
169/169 [==============================] - ETA: 0s - loss: 0.7784 - accuracy: 0.7338

169/169 [==============================] - 3s 17ms/step - loss: 0.7784 - accuracy: 0.7338 - val_loss: 0.7997 - val_accuracy: 0.6797
Epoch 4/5
169/169 [==============================] - ETA: 0s - loss: 0.6592 - accuracy: 0.7837

169/169 [==============================] - 3s 17ms/step - loss: 0.6592 - accuracy: 0.7837 - val_loss: 0.7520 - val_accuracy: 0.6835
Epoch 5/5
169/169 [==============================] - ETA: 0s - loss: 0.5632 - accuracy: 0.8201

83/83 [==============================] - 0s 5ms/step - loss: 0.7199 - accuracy: 0.6850
[0.7199063301086426, 0.6850037574768066]


In [56]:
# Avaliação do modelo
loss, accuracy = model.evaluate(x_test, y_test)
print('Loss:', loss)
print('Accuracy:', accuracy)

83/83 [==============================] - 1s 6ms/step - loss: 0.7199 - accuracy: 0.6850
Loss: 0.7199063301086426
Accuracy: 0.6850037574768066


### Exportação com a biblioteca pickle 

In [57]:
with open('redeNeural_word2vec_embedding.pkl', 'wb') as arquivo:
    pickle.dump(model, arquivo)
with open('redeNeural_word2vec_embedding.pkl', 'rb') as arquivo:
    modelo_redeNeural_word2vec_embedding = pickle.load(arquivo)